In [1]:
cd /mnt/sda/otani/Experiments/moment_retrieval/

/mnt/sda/otani/Experiments/moment_retrieval


In [3]:
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
%matplotlib inline

In [4]:
def load_dataset_file(df_file):
    df = pd.read_csv(df_file)
    indices = np.where(df['end (sec)'] - df['start (sec)'] < 0)
    df = df.drop(index=indices[0])
    df['start (norm)'] = df['start (sec)']/df['length']
    df['end (norm)'] = df['end (sec)']/df['length']
    df['moment duration (norm)'] = df['end (norm)'] - df['start (norm)']
    return df

df = load_dataset_file('data/processed/test.csv')

In [5]:
df.head()

,id,start (sec),end (sec),description,length,start (norm),end (norm),moment duration (norm)
0,3MSZA,24.3,30.4,person turn a light on.,30.96,0.784884,0.981912,0.197028
1,3MSZA,24.3,30.4,person flipped the light switch near the door.,30.96,0.784884,0.981912,0.197028
2,3MSZA,24.3,30.4,person turn the light switch on.,30.96,0.784884,0.981912,0.197028
3,3MSZA,24.3,30.4,person is playing with the switch for the light.,30.96,0.784884,0.981912,0.197028
4,AMT7R,4.3,12.5,a person is putting a picture onto the wall.,30.08,0.142952,0.415559,0.272606


In [6]:
JSON_ROOT = './data/interim/object_detection/charade/'

In [10]:
obj_data = json.load(open(JSON_ROOT+'AMT7R.json'))

In [15]:
try:
    obj_data[121]
except IndexError:
    print('aaa')

aaa


In [102]:
JSON_ROOT = './data/interim/object_detection/charade/'

def get_object_motions(video_id, start_time, end_time):
    detec = json.load(open(JSON_ROOT+video_id+'.json'))
    start_fr = int(np.ceil(4 * start_time))
    end_fr = int(np.ceil(4 * end_time))
    n_fr = end_fr - start_fr
    
    if n_fr == 0:
        return 0
    
    non_background_area = 0
    motions = []
    for i in range(start_fr, end_fr):
        try:
            cur_objs = detec[i]
        except IndexError:
            continue
        
        if len(cur_objs) == 0:
            motions.append([0]) # motion 0 when no objects detected
            continue
        
        try:
            prev_objs = detec[i-1]
        except IndexError:
            motions.append([0])
            continue
        
        m_vals = [] # motions for each object
        for item in cur_objs:
            label = item['label']
            bbox = item['bbox']
            c_y = (bbox[2] + bbox[0]) / 2
            c_x = (bbox[3] + bbox[1]) / 2
            
            m_val = np.inf
            for p_item in prev_objs:
                
                if p_item['label'] != label:
                    continue
                    
                p_bbox = p_item['bbox']
                p_y = (p_bbox[2] + p_bbox[0]) / 2
                p_x = (p_bbox[3] + p_bbox[1]) / 2

                dist = (c_x - p_x)**2 + (c_y - p_y)**2
                dist = np.sqrt(dist)
                if dist < m_val: # select closed object as match
                    m_val = dist
            
            if m_val == np.inf: # if object newly appear at this frame
                m_val = 0
                
            m_vals.append(m_val)
            
        motions.append(m_vals)
    return motions

def get_object_motion_based_score(video_id, start_time, end_time):
    motions = get_object_motions(video_id, start_time, end_time)
    motions = [sum(m) for m in motions]
    score = max(motions)
    return score

In [104]:
scores = []

for _, row in df.iterrows():
    v_id = row['id']
    st = row['start (sec)']
    ed = row['end (sec)']
    s = get_object_motion_based_score(v_id, st, ed)
    scores.append(s)